# TASEP MODEL (NO REATTACHMENT)

### Import dependencies

In [10]:
import math
import numpy as np
import os
import xlsxwriter 
import random
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from queue import Queue
from PIL import Image, ImageDraw, ImageFont
from textwrap import wrap
import glob
from matplotlib.colors import LinearSegmentedColormap
cmap_name = 'my_list'
colors = [(1, 1, 1), (0, 0, 1), (1, 0, 0)]  
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=3) 

### Make image folder

In [11]:
script_dir = os.getcwd()
images_dir = os.path.join(script_dir, 'Images/')
if not os.path.isdir(images_dir):
    os.makedirs(images_dir)
graphs_dir = os.path.join(script_dir, 'Reserve/')
if not os.path.isdir(graphs_dir):
    os.makedirs(graphs_dir)

### Make workbook

In [12]:
workbook = xlsxwriter.Workbook(r'C:\Users\Avirup\Desktop\Pyhton\Output2.xlsx')

worksheet1 = workbook.add_worksheet()              
    
worksheet1.write('A1', "Iteration")
worksheet1.write('B1', "MotorA Input")
worksheet1.write('C1', "MotorB Input")
worksheet1.write('D1', "MotorA Out")
worksheet1.write('E1', "MotorB Out")
worksheet1.write('F1', "MotorA in Reservoir")
worksheet1.write('G1', "MotorB in Reservoir")
print("Done")

Done


### Data




In [13]:
association_rate = 20
rows,cols = 3,100
sites = np.arange(100)
no_a,no_b = 1000,1000 # motor storage
vel_a, vel_b = 2,1 # velocities
pr_a,pr_b = 100,100 # Processivity  
gap = 0
flag = 0
runtime = 1001
reservoir_length = 2000 # reservoir capacity

### Parameters

In [15]:
#Input queue
pool = np.zeros(no_a+no_b)
#Input motors
motor_a = 0   
motor_b = 0       
#Output motors
exit_a = 0 
exit_b = 0 
# Space     
particle = np.zeros((3,100))#state
velocity = np.zeros((3,100))#velocities
span = np.zeros((3,100))    # distance traversed
#Motors lost due to detachment        
detach_a = 0
detach_b = 0 
# Reservoir matrices
reservoir_a = [[0 for j in range(cols)] for i in range(rows)]
reservoir_b = [[0 for j in range(cols)] for i in range(rows)]

for i in range(rows):
    for j in range(cols):
        reservoir_a[i][j] = Queue(maxsize = reservoir_length)
        reservoir_b[i][j] = Queue(maxsize = reservoir_length)
        
waiting_a = [[0 for j in range(cols)] for i in range(rows)] 
waiting_b = [[0 for j in range(cols)] for i in range(rows)] 

productive_a = [[0 for j in range(cols)] for i in range(rows)] 
productive_b = [[0 for j in range(cols)] for i in range(rows)]

 ### Making input array

In [16]:
def motor_input(pool, no_a, no_b, association_rate): # Using the same distribution    

    x = no_a
    y = no_b
    for i in range (no_a+no_b):
        if(x>0 and y>0):
            p=random.random()
            
            if(p>0 and p<=(association_rate/100)): 
                pool[i] = 1
                x-=1
            elif(p>(association_rate/100) and p<=(2*association_rate/100)): 
                pool[i] = 2
                y-=1
            else:
                pool[i] = 0
        elif(x>0 and y==0): # if Bs are exhausted
            pool[i] = 1
            x-=1
        elif(x==0 and y>0): # if As are exhausted
            pool[i] = 2
            y-=1   

### Initial Attachment

In [17]:
def initial(index, pool, rows, motor_a, motor_b):
      
    k = random.randint(0,rows-1)
    particle[k][0] = pool[index]
    index += 1
    # Attaching Motor A
    if(particle[k][0] == 1):
        velocity[k][0] = vel_a  
        span[k][0] = 1
        motor_a += 1        
    #Attaching Motor B
    elif(particle[k][0] == 2):
        velocity[k][0] = vel_b
        span[k][0] = 1
        motor_b += 1          
    
    return index, motor_a, motor_b

### Movement

In [18]:
def movement(time, rows, cols, vel_a, vel_b, pr_a, pr_b, exit_a, exit_b,reservoir_a, reservoir_b,detach_a, detach_b):    
    
    for i in range(rows): # Checking empty sites from last to avoid mentioning that explicitly
        gap,flag=0,0        
        for j in range(cols-1,-1,-1):
            if(particle[i][j] == 0):
                gap += 1
                continue
            else:
                
                if(particle[i][j] == 1):
                    if(gap < vel_a):
                        flag = 0 
                    elif(gap >= vel_a):      
                        flag = vel_a 
                
                
                elif(particle[i][j] == 2):
                    if(gap < vel_b):
                        flag = 0 
                    elif(gap >= vel_b):      
                        flag = vel_b
                                                     
                #Transport
                if(flag > 0): # if empty
                    if(particle[i][j] == 1):
                        if((j + vel_a) >= cols-1):
                            exit_a += 1
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0
                            
                        
                        else:
                            velocity[i][j+flag] = flag
                            particle[i][j+flag] = 1
                            span[i][j+flag] = span[i][j] + flag
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0
                            
                            if(span[i][j+flag] >= pr_a): #exceeds processivity
                               
                                velocity[i][j+flag] = 0
                                particle[i][j+flag] = 0
                                span[i][j+flag] = 0
                                reservoir_a[i][j].put(1)                               
                                detach_a +=1

                    elif(particle[i][j] == 2):
                        if((j + vel_b) >= cols-1):
                            exit_b += 1
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0                           
                        
                        else:
                            velocity[i][j+flag] = flag
                            particle[i][j+flag] = 2
                            span[i][j+flag] = span[i][j] + flag
                            
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0
                            
                            if(span[i][j+flag] >= pr_b):  #exceeds processivity  
                                
                                velocity[i][j+flag] = 0
                                particle[i][j+flag] = 0
                                span[i][j+flag] = 0
                                reservoir_b[i][j].put(2)                                 
                                detach_b +=1
                             
                #Detachment         
                elif(flag == 0):
                    if(particle[i][j] == 1):
                       
                        if(i == 0):
                            lane1 = rows-1   #Lane jump
                        else:
                            lane1 = i-1

                        if(i == rows-1):
                            lane2 = 0
                        else:
                            lane2 = i+1

                        y = random.choice([lane1, lane2])
            
                        if((j + vel_a) >= cols-1): # At last site
                            exit_a += 1
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0
                            
                        else: #LANE JUMP 
                            if(particle[y][j+vel_a] == 0):
                                velocity[y][j+vel_a] = vel_a
                                particle[y][j+vel_a] = 1
                                span[y][j+vel_a] = span[i][j] + vel_a
                                
                                velocity[i][j] = 0
                                particle[i][j] = 0
                                span[i][j] = 0
                                                                
                            else:
                                velocity[i][j] = 0
                                particle[i][j] = 0
                                span[i][j] = 0
                                reservoir_a[i][j].put(1)                                
                                detach_a += 1

                    elif(particle[i][j] == 2):
                        if(i == 0):
                            lane1 = rows-1    #Lane jump
                        else:
                            lane1 = i-1

                        if(i == rows-1):
                            lane2 = 0
                        else:
                            lane2 = i+1

                        y = random.choice([lane1, lane2])
                        
                        if((j + vel_b) >= cols-1): # At last site
                            exit_b += 1
                            velocity[i][j] = 0
                            particle[i][j] = 0
                            span[i][j] = 0                            
                            
                        else:   #LANE JUMP
                            if(particle[y][j+vel_b]== 0):
                                velocity[y][j+vel_b] = vel_b
                                particle[y][j+vel_b] = 2
                                span[y][j+vel_b] = span[i][j] + vel_b
                               
                                velocity[i][j] = 0
                                particle[i][j] = 0
                                span[i][j] = 0
                                                                                                
                            else:
                                velocity[i][j] = 0
                                particle[i][j] = 0
                                span[i][j] = 0                                
                                reservoir_b[i][j].put(2)  
                                detach_b += 1                
                
                gap = 0                   
    
    return exit_a, exit_b,reservoir_a, reservoir_b, detach_a, detach_b

### Main Function

In [19]:
index = 0
motor_input(pool, no_a, no_b, association_rate)

for iter in range(runtime):      

    exit_a, exit_b,reservoir_a, reservoir_b,detach_a, detach_b= movement(iter,rows, cols, vel_a, vel_b, pr_a, pr_b, exit_a, exit_b,reservoir_a, reservoir_b,detach_a,detach_b)
                                               

    if(index < no_a + no_b):
        index, motor_a, motor_b = initial(index,pool, rows, motor_a, motor_b)
    print("Iter",iter)        
    print(motor_a,motor_b)
    print(exit_a,exit_b)
    
    # Writing the values   
    worksheet1.write(iter+1, 0, iter+1)
    worksheet1.write(iter+1, 1, motor_a) 
    worksheet1.write(iter+1, 2, motor_b)
    worksheet1.write(iter+1, 3, exit_a)
    worksheet1.write(iter+1, 4, exit_b)
    worksheet1.write(iter+1, 5, detach_a)
    worksheet1.write(iter+1, 6, detach_b)
    
    for i in range(rows):
        for j in range(cols):
            waiting_a[i][j] = reservoir_a[i][j].qsize()
            waiting_b[i][j] = reservoir_b[i][j].qsize()
            productive_a[i][j] += waiting_a[i][j]
            productive_b[i][j] += waiting_b[i][j]           
                      
    print("Done")     
        
    if(iter>0 and iter%20==0):
               
        no = str(iter)        
        #Plotting the state        
        time=iter*0.04        
        plt.pcolor(particle, linewidths = 0.02, cmap=cm)
        plt.title("\n".join(wrap("At time %0.2f"%time + ' secs', 60)), fontsize=12)
        plt.xlabel('Sites')
        plt.ylabel('Track')
        custom_lines = [Patch(facecolor='white', edgecolor='k',label='Empty Space'),Patch(facecolor='blue', edgecolor='k',label='Motor A'),
                        Patch(facecolor='red', edgecolor='k',label='Motor B')]
        plt.legend(handles=custom_lines,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xlim([0, 100])
        plt.yticks(np.arange(0, 4, 1))
        plt.gcf().text(.1, 0.01, "Data Values:", fontsize=10)
        plt.gcf().text(.1, -0.04, "Association rate: 10 motors/sec", fontsize=8) #0.4motors/iter and 1 iter = 0.04s
        plt.gcf().text(.1, -0.08, "Input::     Motor A: " +str(motor_a) + "     Motor B: " +str(motor_b), fontsize=8)
        plt.gcf().text(.1, -0.12, "Output::  Motor A: " +str(exit_a) + "     Motor B: " +str(exit_b), fontsize=8)
        plt.gcf().text(.1, -0.16, "Reservoir::         Motor A: " +str(detach_a) + "     Motor B: " +str(detach_b), fontsize=8)
        plt.tight_layout()
        plt.savefig('Images/State_'+no+'.png',  dpi=100, bbox_inches = 'tight')
        plt.draw()
        plt.close()
        print("Graph Done")
        #Plotting Detached Motors for each site in space
        no = str(iter)             
        
        wait1 = [[0 for j in range(cols)] for i in range(rows)]
        wait2 = [[0 for j in range(cols)] for i in range(rows)] 
                
        for i in range(rows):
            for j in range(cols):
               
                wait1[i][j] = productive_a[i][j]/iter
                wait2[i][j] = productive_b[i][j]/iter
                                
                
        plt.figure(1)
        fig, (ax0, ax1, ax2) = plt.subplots(3, 1)
        #Track 1
        ax0.bar(sites, wait1[0], alpha=0.6, color="Blue", label="Motor A")
        ax0.bar(sites, wait2[0], alpha=0.5, color="Red", label="Motor B")
        ax0.set_xlabel('Site Number', fontsize=15)
        ax0.set_ylabel("\n".join(wrap('No of Motors', 22)), fontsize=15)
        ax0.set_title('Track 1', fontsize=15)
        ax0.legend(loc="upper right")
        ax0.set_xlim([0, 100])
        #Track 2
        ax1.bar(sites, wait1[1], alpha=0.6, color="Blue", label="Motor A")
        ax1.bar(sites, wait2[1], alpha=0.5, color="Red", label="Motor B")
        ax1.set_xlabel('Site Number', fontsize=15)
        ax1.set_ylabel("\n".join(wrap('No of Motors', 22)), fontsize=15)
        ax1.set_title('Track 2', fontsize=15)
        ax1.legend(loc="upper right")
        ax1.set_xlim([0, 100])
        #Track 3
        ax2.bar(sites, wait1[2], alpha=0.6, color="Blue", label="Motor A")
        ax2.bar(sites, wait2[2], alpha=0.5, color="Red", label="Motor B")
        ax2.set_xlabel('Site Number', fontsize=15)
        ax2.set_ylabel("\n".join(wrap('No of Motors', 22)), fontsize=15)
        ax2.set_title('Track 3', fontsize=15)
        ax2.legend(loc="upper right")
        ax2.set_xlim([0, 100])

        fig.suptitle("\n".join(wrap('Motors in Productive Reservoirs after time %.2f'%time + ' secs', 45)), fontsize=16)
        fig.tight_layout()
        plt.subplots_adjust(bottom=-0.5, right=0.8, top=0.7, hspace = 1.2)
        plt.gcf()
        plt.savefig('Reserve/Detached motors_'+no+'.png', bbox_inches = 'tight')
        plt.close()
                       
workbook.close()

Iter 0
0 0
0 0
Done
Iter 1
1 0
0 0
Done
Iter 2
1 0
0 0
Done
Iter 3
2 0
0 0
Done
Iter 4
3 0
0 0
Done
Iter 5
3 0
0 0
Done
Iter 6
3 0
0 0
Done
Iter 7
3 1
0 0
Done
Iter 8
4 1
0 0
Done
Iter 9
4 1
0 0
Done
Iter 10
4 2
0 0
Done
Iter 11
5 2
0 0
Done
Iter 12
5 2
0 0
Done
Iter 13
5 2
0 0
Done
Iter 14
5 2
0 0
Done
Iter 15
5 3
0 0
Done
Iter 16
5 4
0 0
Done
Iter 17
5 5
0 0
Done
Iter 18
5 5
0 0
Done
Iter 19
6 5
0 0
Done
Iter 20
6 5
0 0
Done
Graph Done
Iter 21
6 5
0 0
Done
Iter 22
6 5
0 0
Done
Iter 23
6 6
0 0
Done
Iter 24
6 6
0 0
Done
Iter 25
6 6
0 0
Done
Iter 26
6 6
0 0
Done
Iter 27
7 6
0 0
Done
Iter 28
8 6
0 0
Done
Iter 29
8 6
0 0
Done
Iter 30
8 7
0 0
Done
Iter 31
9 7
0 0
Done
Iter 32
9 7
0 0
Done
Iter 33
9 7
0 0
Done
Iter 34
9 7
0 0
Done
Iter 35
10 7
0 0
Done
Iter 36
11 7
0 0
Done
Iter 37
11 7
0 0
Done
Iter 38
12 7
0 0
Done
Iter 39
12 7
0 0
Done
Iter 40
12 7
0 0
Done
Graph Done
Iter 41
12 7
0 0
Done
Iter 42
12 7
0 0
Done
Iter 43
13 7
0 0
Done
Iter 44
13 7
0 0
Done
Iter 45
13 7
0 0
Done
Iter 46
13 

Graph Done
Iter 341
75 55
51 34
Done
Iter 342
76 55
51 34
Done
Iter 343
76 56
51 34
Done
Iter 344
76 57
51 34
Done
Iter 345
76 57
51 34
Done
Iter 346
76 57
51 34
Done
Iter 347
77 57
51 34
Done
Iter 348
77 58
51 34
Done
Iter 349
77 59
51 34
Done
Iter 350
77 59
51 34
Done
Iter 351
77 59
51 34
Done
Iter 352
78 59
51 34
Done
Iter 353
78 59
51 34
Done
Iter 354
79 59
52 34
Done
Iter 355
79 60
52 34
Done
Iter 356
79 60
52 34
Done
Iter 357
79 61
52 34
Done
Iter 358
80 61
52 35
Done
Iter 359
80 62
52 35
Done
Iter 360
80 63
52 35
Done
Graph Done
Iter 361
81 63
52 35
Done
Iter 362
82 63
52 36
Done
Iter 363
82 63
53 36
Done
Iter 364
82 63
53 36
Done
Iter 365
82 63
53 36
Done
Iter 366
82 64
53 36
Done
Iter 367
82 64
53 36
Done
Iter 368
82 64
53 36
Done
Iter 369
82 64
53 36
Done
Iter 370
82 64
53 37
Done
Iter 371
83 64
53 38
Done
Iter 372
83 64
53 38
Done
Iter 373
83 64
53 38
Done
Iter 374
83 65
53 38
Done
Iter 375
83 65
53 38
Done
Iter 376
83 65
53 38
Done
Iter 377
83 66
54 38
Done
Iter 378
83 67
5

Graph Done
Iter 641
138 113
86 82
Done
Iter 642
138 113
86 82
Done
Iter 643
138 113
86 83
Done
Iter 644
139 113
86 83
Done
Iter 645
139 113
86 83
Done
Iter 646
139 113
86 84
Done
Iter 647
140 113
86 85
Done
Iter 648
140 114
86 85
Done
Iter 649
140 114
86 85
Done
Iter 650
140 115
86 85
Done
Iter 651
141 115
86 85
Done
Iter 652
141 115
86 85
Done
Iter 653
141 115
86 85
Done
Iter 654
141 115
86 85
Done
Iter 655
142 115
86 85
Done
Iter 656
142 115
86 85
Done
Iter 657
142 115
87 85
Done
Iter 658
142 116
87 85
Done
Iter 659
143 116
87 87
Done
Iter 660
144 116
87 87
Done
Graph Done
Iter 661
144 117
87 88
Done
Iter 662
144 117
87 88
Done
Iter 663
144 118
87 88
Done
Iter 664
144 118
87 88
Done
Iter 665
145 118
87 88
Done
Iter 666
145 118
88 88
Done
Iter 667
145 118
88 88
Done
Iter 668
145 119
88 88
Done
Iter 669
145 119
88 88
Done
Iter 670
146 119
88 88
Done
Iter 671
147 119
88 88
Done
Iter 672
147 119
88 88
Done
Iter 673
148 119
88 88
Done
Iter 674
148 119
88 88
Done
Iter 675
148 120
88 88
Don

Graph Done
Iter 921
199 170
118 134
Done
Iter 922
200 170
118 134
Done
Iter 923
201 170
118 134
Done
Iter 924
201 170
119 134
Done
Iter 925
201 170
119 134
Done
Iter 926
201 171
119 135
Done
Iter 927
201 171
119 135
Done
Iter 928
201 171
120 135
Done
Iter 929
201 171
120 135
Done
Iter 930
202 171
120 136
Done
Iter 931
202 171
120 136
Done
Iter 932
202 172
120 137
Done
Iter 933
202 172
120 137
Done
Iter 934
202 173
121 137
Done
Iter 935
202 174
121 137
Done
Iter 936
202 174
121 137
Done
Iter 937
202 174
121 137
Done
Iter 938
202 174
123 137
Done
Iter 939
202 175
123 137
Done
Iter 940
203 175
123 137
Done
Graph Done
Iter 941
203 176
123 137
Done
Iter 942
203 177
123 137
Done
Iter 943
203 177
123 137
Done
Iter 944
203 178
123 138
Done
Iter 945
204 178
123 138
Done
Iter 946
204 178
123 138
Done
Iter 947
204 178
123 139
Done
Iter 948
204 178
123 139
Done
Iter 949
204 178
123 139
Done
Iter 950
204 178
123 139
Done
Iter 951
204 178
123 140
Done
Iter 952
204 179
123 140
Done
Iter 953
205 179
1

<Figure size 432x288 with 0 Axes>